## Loading data 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os
import re
from pathlib import Path

In [ ]:
data = pd.read_csv('complaints.csv')

## Load .msg files as below

In [3]:
# import extract_msg
# data_path = Path('folder')

In [4]:
# pattern1 = re.compile(r'From: .*')
# pattern2 = re.compile(r'Sent: .*')
# pattern3 = re.compile(r'To: .*')
# pattern4 = re.compile(r'[\n\r]')
# pattern5 = re.compile(r'\d+')
# pattern6 = re.compile(r'\S+@\S+')
# pattern7 = re.compile(r'[!"#$%&\'()*+,-./:;<=>?@\[\\\]^_`{|}~]')
# patterns = [pattern1, pattern2, pattern3, pattern4, pattern5, pattern6, pattern7]
# pattern_a = re.compile(r'Cc: (.*)')

# def remove_pattern(*patterns, text, group=0):
#     '''
#     :param patterns: list of patterns where each patter is a re.compile
#     :param text: string, text
#     :return: text, string, having all the patterns removed
#     '''
#     for pattern in patterns:
#         matches = pattern.finditer(text)
#         temp = ''
#         pos_prev = 0
#         pos_cur = 0
#         for match in matches:
#             pos_cur = match.span(0)[0]
#             temp += text[pos_prev: pos_cur]
#             temp += ' '
#             pos_prev = match.span(0)[1]
#         temp += text[pos_prev:]
#         if temp != '':
#             text = temp
#     return text

In [5]:
# file_names = os.listdir(data_path)

# data = pd.DataFrame(columns=['file_name', 'message'], index=range(len(file_names)))
# for i, file_name in enumerate(file_names):
#     file_path = data_path / file_name
#     msg = extract_msg.Message(file_path)
#     msg_message = msg.body
#     msg_message = remove_pattern(*[pattern_a], text=msg_message, group=1)
#     msg_message = remove_pattern(*patterns, text=msg_message)
#     data.iloc[i] = file_name, msg_message

In [6]:
data.shape

(179776, 2)

In [7]:
data.head(2)

,Consumer complaint narrative,Product
0,I have outdated information on my credit repor...,Credit reporting
1,I purchased a new car on XXXX XXXX. The car de...,Consumer Loan


In [8]:
data = data.dropna(axis=0, subset=['Consumer complaint narrative']).reset_index(drop=True)
data.shape

(179776, 2)

### Typical Complaint

In [9]:
data.Product.value_counts()

Debt collection                                                                 43946
Mortgage                                                                        34576
Credit reporting                                                                31589
Credit card                                                                     18845
Bank account or service                                                         14892
Student loan                                                                    12047
Consumer Loan                                                                    9486
Credit reporting, credit repair services, or other personal consumer reports     6118
Payday loan                                                                      1750
Credit card or prepaid card                                                      1518
Money transfers                                                                  1496
Prepaid card                                          

In [10]:
def get_data(cols, ratio, n):
    global data
    new_data = pd.DataFrame(columns=['Product', 'Consumer complaint narrative'])
    for i, col in enumerate(cols):
        a = int(n * ratio[i] / sum(ratio))
        temp_data = data[data['Product'] == col].reset_index(drop=True)
        for j in range(a):
            new_data = new_data.append(temp_data.iloc[j, :])
        new_data = new_data.iloc[np.random.choice(a=new_data.shape[0], size=new_data.shape[0], replace=False), :]
    return new_data

In [11]:
data = get_data(cols=['Debt collection', 'Mortgage', 'Credit reporting'], ratio=[1, 1, 1], n=600)

In [12]:
data.shape

(600, 2)

In [13]:
data.head()

,Product,Consumer complaint narrative
115,Mortgage,I lost my job XXXX 2016. Was on unemployment. ...
105,Debt collection,I went through a credit consolidation company ...
158,Mortgage,"We are subject yet again, to more delays from ..."
86,Credit reporting,"XXXX, Equifax, and XXXX placed fraud alerts on..."
70,Credit reporting,Equifax is not responding to my request to inv...


In [14]:
print(data['Product'].isna().sum())
print(data['Consumer complaint narrative'].isna().sum())

0
0


### preprocessing text data

In [15]:
x = list(data['Consumer complaint narrative'].values)
y = list(data['Product'].values)

In [16]:
# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer = CountVectorizer()
# vectorizer.fit(x)
# vocab = vectorizer.vocabulary_
# vocab = dict(sorted(vocab.items(), key=lambda item: item[1], reverse=True))

In [17]:
x[58]

'On XX/XX/XXXX the state of XXXX filed a tax lien against me. This lien has been reflecting on my XXXX and Transunion credit reports for the past several years. \n\nI am an XXXX, between XX/XX/XXXX and XX/XX/XXXX I was XXXX in XXXX. During that time I had moved to XXXX different residences due to work. I did not receive notice of delinquent taxes during my XXXX in that state. Nor did I receive notification of delinquent taxes after moving to XXXX on XXXX in XX/XX/XXXX. \n\nAt this point I do not have tax documentation from XX/XX/XXXX through XX/XX/XXXX and will gladly pay the debt to satisfy the issue as it is unnecessarily damaging my credit. \n\nI never received proper notification of the debt, and am writing to kindly request the tax lien be withdrawn, and I be allowed the opportunity to pay the debt. In addition I ask that XXXX, and Transunion remove that negative information from my credit report. \n\nI have contacted the XXXX State Department of Assessments & Taxation in an attem

In [18]:
pattern = re.compile(r'[\n]')

In [19]:
for i in range(len(x)):
    x[i] = re.sub(pattern, ' ', x[i])

In [20]:
x[58]

'On XX/XX/XXXX the state of XXXX filed a tax lien against me. This lien has been reflecting on my XXXX and Transunion credit reports for the past several years.   I am an XXXX, between XX/XX/XXXX and XX/XX/XXXX I was XXXX in XXXX. During that time I had moved to XXXX different residences due to work. I did not receive notice of delinquent taxes during my XXXX in that state. Nor did I receive notification of delinquent taxes after moving to XXXX on XXXX in XX/XX/XXXX.   At this point I do not have tax documentation from XX/XX/XXXX through XX/XX/XXXX and will gladly pay the debt to satisfy the issue as it is unnecessarily damaging my credit.   I never received proper notification of the debt, and am writing to kindly request the tax lien be withdrawn, and I be allowed the opportunity to pay the debt. In addition I ask that XXXX, and Transunion remove that negative information from my credit report.   I have contacted the XXXX State Department of Assessments & Taxation in an attempt to re

### Train-test split
15% of the total data is used as validation data while the remaining as training.

In [21]:
x_train_text, x_test_text, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=123)
print('Training utterances: {}'.format(len(x_train_text)))
print('Validation utterances: {}'.format(len(x_test_text)))

Training utterances: 450
Validation utterances: 150


In [22]:
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)
print(y_train_enc)

[2 1 0 1 2 0 0 2 1 0 1 2 2 2 2 0 2 2 0 1 0 0 2 2 1 0 0 1 1 1 0 1 2 1 1 1 1
 1 0 0 2 0 0 0 2 0 2 2 1 1 1 1 2 0 1 2 1 0 2 0 1 1 0 0 2 1 0 2 2 2 0 2 0 1
 0 0 1 1 2 1 0 0 0 1 0 1 2 2 2 2 1 1 1 0 1 0 1 0 2 2 1 2 1 0 2 0 0 2 1 0 0
 0 2 1 0 2 2 0 2 0 1 2 0 1 2 0 1 2 1 1 2 2 1 1 2 1 0 1 0 0 0 0 0 2 2 1 2 0
 1 2 0 2 0 0 2 0 0 1 0 0 1 1 1 0 1 2 1 1 2 0 1 2 0 0 1 1 0 2 2 0 2 2 1 2 2
 2 0 0 1 2 1 1 0 2 0 1 0 0 1 2 0 2 2 2 0 0 0 2 1 1 2 0 0 2 2 0 2 0 1 0 0 1
 1 1 2 2 2 0 0 2 2 1 1 1 1 0 0 1 2 0 2 1 2 0 1 2 1 1 1 0 2 1 1 2 0 0 0 2 1
 1 1 2 2 2 1 0 2 0 1 0 2 0 0 2 0 0 0 1 1 1 2 0 0 1 2 0 1 1 2 2 1 2 0 0 0 1
 1 0 0 0 0 2 2 1 1 2 0 0 1 0 1 0 0 1 0 2 1 0 2 1 1 0 1 2 2 2 1 2 2 0 0 0 2
 1 1 0 2 1 0 1 2 0 2 1 1 2 0 0 1 2 2 1 1 1 0 2 2 0 1 0 2 0 0 2 2 2 0 0 0 2
 0 2 1 1 2 0 2 1 1 1 0 2 1 0 1 1 0 2 2 2 2 1 2 2 1 2 1 0 0 1 0 2 1 1 2 1 0
 1 2 0 0 2 1 2 2 2 1 2 0 1 1 1 1 2 1 2 1 2 2 0 0 0 2 0 2 0 2 2 0 1 1 1 0 1
 0 0 2 0 2 2]


In [23]:
num_classes = len(np.unique(y_train))

## Modelling

In [1]:
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

In [25]:
mean = np.mean([len(text.split()) for text in x_train_text])
std = np.std([len(text.split()) for text in x_train_text])
max_length = int(std + mean)
max_length

347

In [26]:
# save model data
import pickle
if not os.path.exists('model_data'):
    os.mkdir('model_data')
pickle.dump(le, open(Path('model_data') / 'le.pickle', 'wb'))
pickle.dump(max_length, open(Path('model_data') / 'max_length.pickle', 'wb'))

if not os.path.exists('saved_models'):
    os.mkdir('saved_models')

Note: In case of Resource exhausted error, lower the batch_size, which might result in loss of performance

### DistillBERT

In [27]:
from transformers import TFDistilBertModel, DistilBertConfig, DistilBertTokenizerFast

In [28]:
# Name of the BERT model to use
model_name = 'distilbert-base-uncased'

# Load transformers config and set number of classes
config = DistilBertConfig.from_pretrained(model_name)
config.num_labels = num_classes

# Load BERT tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained(pretrained_model_name_or_path=model_name, config=config)

# Load the base BERT model
distilbert_model = TFDistilBertModel.from_pretrained(model_name, config=config)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'vocab_layer_norm', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [29]:
# Tokenize texts 
def tokenize(text):
    tokenized = tokenizer(text,
                          max_length=max_length,
                          padding=True,
                          truncation=True,
                          return_tensors='tf',
                          return_token_type_ids=False,
                          return_attention_mask=False,
                          verbose=True)
    return tokenized

x_train_enc = tokenize(x_train_text)
x_test_enc = tokenize(x_test_text)

y_train_enc_one_hot = to_categorical(y_train_enc, num_classes=num_classes)
y_test_enc_one_hot = to_categorical(y_test_enc, num_classes=num_classes)

In [30]:
# define layers (head-layers)
distilbert = distilbert_model.layers[0]
pre_classifier = Dense(units=config.dim,
                       kernel_initializer=TruncatedNormal(stddev=config.initializer_range),
                       activation='relu',
                       name='pre_classifier')
classifier = Dense(units=config.num_labels,
                   kernel_initializer=TruncatedNormal(stddev=config.initializer_range),
                   name='classifier')
dropout = Dropout(config.seq_classif_dropout)

# Define model input
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}

# Forward pass
distilbert_output = distilbert(inputs)
hidden_state = distilbert_output[0]
pooled_output = hidden_state[:, 0]
pooled_output = pre_classifier(pooled_output)
pooled_output = dropout(pooled_output) # , training=False True or False?
logits = classifier(pooled_output) 

# complete a model using TF functional API
model = Model(inputs=inputs, outputs=logits, name='Complaint_Classifier')

# Model summary
model.summary()

Model: "Complaint_Classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 347)]             0         
_________________________________________________________________
distilbert (TFDistilBertMain ((None, 347, 768),)       66362880  
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
pre_classifier (Dense)       (None, 768)               590592    
_________________________________________________________________
dropout_19 (Dropout)         (None, 768)               0         
_________________________________________________________________
classifier (Dense)           (None, 3)                 2307      
Total params: 66,955,779
Trainable params: 66,955,779
Non-trainable params: 0
__________________________________

In [31]:
lr = 1e-05
bs = 8
# set callbacks
cb = [ModelCheckpoint(filepath='saved_models/'+'model.{epoch:02d}-{accuracy:02f}-{val_accuracy:.2f}',
                      monitor='val_accuracy',
                      mode='max',
                      save_best_only=True)] # callbacks

In [32]:
### ------- Train the model ------- ###
# Set an optimizer
optimizer = Adam(learning_rate=lr)

# Set loss and metrics
loss = CategoricalCrossentropy(from_logits=True)
metric = CategoricalAccuracy('accuracy')

# Compile the model
model.compile(
    optimizer=optimizer,
    loss=loss, 
    metrics=metric)

# Fit the model
history = model.fit(
    x={'input_ids': x_train_enc['input_ids']},
    y=y_train_enc_one_hot,
    validation_data=({'input_ids': x_test_enc['input_ids']}, y_test_enc_one_hot),
    batch_size=bs,
    epochs=10,
    callbacks=cb)

Epoch 1/10
57/57 [==============================] - ETA: 0s - loss: 1.0671 - accuracy: 0.4489WARNING:tensorflow:Skipping full serialization of Keras layer <tensorflow.python.keras.layers.core.Dropout object at 0x7fbffc13e150>, because it is not built.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_models/model.01-0.448889-0.55/assets
57/57 [==============================] - 47s 831ms/step - loss: 1.0671 - accuracy: 0.4489 - val_loss: 0.9787 - val_accuracy: 0.5467
Epoch 2/10
57/57 [==============================] - ETA: 0s - loss: 0.7883 - accuracy: 0.7400WARNING:tensorflow:Skipping full serialization of Keras layer <tensorflow.python.keras.layers.core.Dropout object at 0x7fbffc13e150>, because it is not built.
INFO:tensorflow:Assets written to: saved_models/mode

In [ ]:
# Save the model
# model.save()

## XLNet

In [ ]:
from transformers import XLNetTokenizerFast, TFXLNetMainLayer, TFSequenceSummary
from transformers import TFXLNetModel, XLNetConfig

In [ ]:
# Name of the pre-trained model to use
model_name = 'xlnet-base-cased'

# Load transformers config and set number of classes
config = XLNetConfig.from_pretrained(model_name)
config.num_labels = num_classes

# Load XLNet tokenizer
tokenizer = XLNetTokenizerFast.from_pretrained(pretrained_model_name_or_path=model_name, config=config)

# Load the XLNet base model
xlnet_model = TFXLNetModel.from_pretrained(model_name, config=config)

In [ ]:
# Prepare input data
# Tokenize texts 
def tokenize(text):
    tokenized = tokenizer(text,
              padding=True,
              truncation=True,
              return_tensors='tf',
              return_token_type_ids=False,
              return_attention_mask=False,
              verbose=True)
    return tokenized

x_train_enc = tokenize(x_train_text)
x_test_enc = tokenize(x_test_text)

y_train_enc_one_hot = to_categorical(y_train_enc, num_classes=num_classes)
y_test_enc_one_hot = to_categorical(y_test_enc, num_classes=num_classes)

In [ ]:
# define layers (head-layers)
transformer = TFXLNetMainLayer(config, name="transformer")
sequence_summary = TFSequenceSummary(config,
                                     initializer_range=config.initializer_range, 
                                     name="sequence_summary")
logits_proj = Dense(config.num_labels,
                    kernel_initializer=TruncatedNormal(stddev=config.initializer_range),
                    name="logits_proj")

# define model input
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}

# forward pass
transformer_outputs = xlnet_model(inputs)
output = transformer_outputs[0]
output = sequence_summary(output)
logits = logits_proj(output)

# complete a model using TF functional API
model = Model(inputs=inputs, outputs=logits, name='Text_Classifier')

# model summary
model.summary()

In [ ]:
lr = 5e-5
bs = 16
# set callbacks
cb = [ModelCheckpoint(filepath='model.{epoch:02d}-{accuracy:02f}-{val_accuracy:.2f}',
                      monitor='val_accuracy',
                      mode='max',
                      save_best_only=True)] # callbacks

In [ ]:
### ------- Train the model ------- ###
# Set an optimizer
optimizer = Adam(learning_rate=lr)
# Set loss and metrics
loss = CategoricalCrossentropy(from_logits=True)
metric = CategoricalAccuracy('accuracy')

# Compile the model
model.compile(
    optimizer=optimizer,
    loss=loss, 
    metrics=metric)

# Fit the model
history = model.fit(
    x={'input_ids': x_train_enc['input_ids']},
    y=y_train_enc_one_hot,
    validation_data=({'input_ids': x_test_enc['input_ids']}, y_test_enc_one_hot),
    batch_size=bs,
    epochs=2,
    callbacks=cb)

In [ ]:
# Save the model
# model.save()

## Results

In [ ]:
def get_preds(x_enc):
# inp = tokenizer(x, truncation=True, padding=True)
    inp_ids = np.asarray(x_enc['input_ids'])
    # att_masks = np.asarray(x_enc['attention_mask'])
    preds = np.empty(inp_ids.shape[0], dtype='uint8')
    for i in range(inp_ids.shape[0]):
        inp_id = inp_ids[i: i+1, :]
        # att_mask = att_masks[i: i+1, :]
        # pred = model(inputs=inp_id, attention_mask=att_mask)
        pred = model(inputs=inp_id)[0]
        pred = tf.math.argmax(pred)
        pred = pred.numpy()
        preds[i] = pred
    return preds

def get_pred_from_text(txt):
    ''' txt: list of texts
    '''
    pred = tokenize(txt)
    pred = get_preds(pred)
    return pred

def get_score(y_true, y_pred):
    ''' y_true, y_pred: type: numpy-array
    '''
    return sum(y_pred == y_true)/len(y_pred)*100

In [ ]:
# n = 200
test = x_train_text
y = np.asarray(y_train_enc)

y_pred = get_pred_from_text(test)
print(get_score(y, y_pred))

In [ ]:
test = x_test_text
y = np.asarray(y_test_enc)

y_pred = get_pred_from_text(test)
print(get_score(y, y_pred))